# Importing libraries

In [229]:
import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
import matplotlib.pyplot as plt
%matplotlib inline
import scipy

import missingno as msno
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Features by data type

In [165]:
### features_list selects which features to include.
features_list = ['poi', 'salary', 'deferral_payments', 'total_payments', 'loan_advances','bonus', 
                      'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 
                      'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 
                      'restricted_stock', 'director_fees', 'to_messages', 'from_poi_to_this_person', 'from_messages', 
                       'from_this_person_to_poi', 'shared_receipt_with_poi'
                ]

# Identifying columns with financial values
financial_features = ['salary', 'deferral_payments', 'total_payments', 'loan_advances','bonus', 
                      'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 
                      'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 
                      'restricted_stock', 'director_fees'
                     ]

# Identfying columns with numerical values
features_with_count = ['to_messages', 'from_poi_to_this_person', 'from_messages', 
                       'from_this_person_to_poi', 'shared_receipt_with_poi'
                      ]

In [166]:
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

# Removing the 'TOTAL' value in data_dict because it is a column sum of salaries and doesn't belong to any single employee.
del data_dict['TOTAL']

### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

# EDA


In [167]:
# Transforming features into a df so that I won't have to remember to transform both features train and test.
df_features = pd.DataFrame(features)
print "Any null-values present in the features chosen? \n", df_features.isnull().any()
df_features.head()

Any null-values present in the features chosen? 
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
dtype: bool


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,201955.0,2869717.0,4484442.0,0.0,4175000.0,-126027.0,-3081055.0,1729541.0,13868.0,1729541.0,152.0,304805.0,126027.0,0.0,2902.0,47.0,2195.0,65.0,1407.0
1,0.0,178980.0,182466.0,0.0,0.0,0.0,0.0,257817.0,3486.0,257817.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,477.0,0.0,916197.0,0.0,0.0,-560222.0,-5104.0,5243487.0,56301.0,4046157.0,864523.0,0.0,1757552.0,0.0,566.0,39.0,29.0,0.0,465.0
3,267102.0,1295738.0,5634343.0,0.0,1200000.0,0.0,-1386055.0,10623258.0,11200.0,6680544.0,2660303.0,1586055.0,3942714.0,0.0,0.0,0.0,0.0,0.0,0.0
4,239671.0,260455.0,827696.0,0.0,400000.0,-82782.0,-201641.0,63014.0,129142.0,0.0,69.0,0.0,145796.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
# Splitting dataset into train and test for features and labels
features_train, features_test, labels_train, labels_test = \
    train_test_split(df_features, labels, test_size=0.3, random_state=42)

# Feature importances using Decision Tree

In [169]:
################
# DecisionTree #
################

# Fitting the model
clf = tree.DecisionTreeClassifier(random_state=42)
clf.fit(features_train, labels_train)

# Feature Importances to identify which features have a high variance to be included in final model and which to exclude.
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
print "Feature Ranking: "
for i in range(len(importances)):
    print "  {}.  feature: {} ({})".format(i+1, features_list[i+1], importances[indices[i]])

Feature Ranking: 
  1.  feature: salary (0.220426513942)
  2.  feature: deferral_payments (0.21197488041)
  3.  feature: total_payments (0.132625994695)
  4.  feature: loan_advances (0.106100795756)
  5.  feature: bonus (0.105863661155)
  6.  feature: restricted_stock_deferred (0.0757862826828)
  7.  feature: deferred_income (0.0736811081639)
  8.  feature: total_stock_value (0.0620731020005)
  9.  feature: expenses (0.0114676611954)
  10.  feature: exercised_stock_options (0.0)
  11.  feature: other (0.0)
  12.  feature: long_term_incentive (0.0)
  13.  feature: restricted_stock (0.0)
  14.  feature: director_fees (0.0)
  15.  feature: to_messages (0.0)
  16.  feature: from_poi_to_this_person (0.0)
  17.  feature: from_messages (0.0)
  18.  feature: from_this_person_to_poi (0.0)
  19.  feature: shared_receipt_with_poi (0.0)


# Feature engineering

In [242]:
# New feature is bonus/salary and null-values replaced by 0
features_train['bonus_salary_ratio'] = features_train.loc[:, 5] / features_train.loc[:, 1]
features_train['bonus_salary_ratio'] = np.nan_to_num(features_train['bonus_salary_ratio'])

# Repeat same feature engineering for test data
features_test['bonus_salary_ratio'] = features_test.loc[:, 5] / features_test.loc[:, 1]
features_test['bonus_salary_ratio'] = np.nan_to_num(features_test['bonus_salary_ratio'])

features_train.head()

# New feature proved to make my models perform worse. See explanation at the Q&A section at the end

/Users/ammarjawaddoosh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ammarjawaddoosh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ammarjawaddoosh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,bonus_salary_ratio
28,314288.0,0.0,1101393.0,0.0,800000.0,0.0,-41250.0,495633.0,27861.0,117551.0,494.0,0.0,378082.0,0.0,102.0,0.0,33.0,4.0,71.0,0.000000e+00
40,182245.0,0.0,2692324.0,0.0,200000.0,0.0,0.0,1008941.0,21530.0,601438.0,53775.0,2234774.0,407503.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,758931.0,0.0,664375.0,0.0,0.0,94556.0,0.0,1433.0,25.0,215.0,2.0,508.0,0.000000e+00
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6077885.0,0.0,5127155.0,0.0,0.0,950730.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
25,0.0,0.0,0.0,0.0,0.0,-472568.0,0.0,189518.0,0.0,0.0,0.0,0.0,662086.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.797693e+308


## Refining feature_list based on feature importances

In [237]:
feature_list = ['poi', 'salary', 'deferral_payments', 'total_payments', 
                'loan_advances', 'bonus', 'restricted_stock_deferred',
                'deferred_income', 'total_stock_value', 'expenses', 'bonus_salary_ratio'
               ]

In [243]:
######################
# KNearest Neighbour #
######################

clf = KNeighborsClassifier(p=1, weights='distance', leaf_size=19, algorithm='ball_tree', n_neighbors=2)

# clf_tree_rscv = RandomizedSearchCV(clf, param_distributions=params, cv=70, n_iter=100, scoring='recall', n_jobs=-1, verbose=2)
# clf_tree_rscv.fit(features_train, labels_train)
# print clf_tree_rscv.best_params_
# print clf_tree_rscv.best_score_

clf.fit(features_train_scaled, labels_train)
pred = clf.predict(features_test_scaled)

# Classifier scores
precision_score_knn = precision_score(pred, labels_test)
recall_score_knn = recall_score(pred, labels_test)
accuracy_score_knn = accuracy_score(pred, labels_test)

print "Accuracy score: ", accuracy_score_tree
print "Precision score: ", precision_score_tree
print "Recall score: ", recall_score_tree

# {'p': 1, 'weights': 'distance', 'leaf_size': 19, 'algorithm': 'ball_tree', 'n_neighbors': 2}
# 0.09

# Dumping classifier, my_dataset and features_list as .pkl files to be used in tester.py
dump_classifier_and_data(clf, my_dataset, features_list)

Accuracy score:  0.613636363636
Precision score:  0.6
Recall score:  0.166666666667


In [228]:
#################
# Decision Tree #
#################

clf = tree.DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=17, max_features=18, class_weight='balanced', splitter='random', min_samples_leaf=14, min_samples_split=63)
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)


# clf_tree_rscv = RandomizedSearchCV(clf, param_distributions=parameters, cv=70, n_iter=80, scoring='precision', n_jobs=-1, verbose=2)
# clf_tree_rscv.fit(features_train, labels_train)

# print clf_tree_rscv.best_params_
# print clf_tree_rscv.best_score_

# {'splitter': 'random', 'min_samples_leaf': 14, 'max_features': 18, 'criterion': 'gini', 'min_samples_split': 49, 'max_depth': 17, 'class_weight': 'balanced'}
# 0.25

# {'min_samples_split': 63}
# 0.25

# Classifier scores
precision_score_tree = precision_score(pred, labels_test)
recall_score_tree = recall_score(pred, labels_test)
accuracy_score_tree = accuracy_score(pred, labels_test)

print "Accuracy score: ", accuracy_score_tree
print "Precision score: ", precision_score_tree
print "Recall score: ", recall_score_tree

# Dumping classifier, my_dataset and features_list as .pkl files to be used in tester.py
dump_classifier_and_data(clf, my_dataset, features_list)

Accuracy score:  0.613636363636
Precision score:  0.6
Recall score:  0.166666666667


In [244]:
###############
# Naive Bayes #
###############

clf = GaussianNB()
clf.fit(features_train, labels_train)

pred = clf.predict(features_test)

# Classifier scores
precision_score_gnb = precision_score(pred, labels_test)
recall_score_gnb = recall_score(pred, labels_test)
accuracy_score_gnb = accuracy_score(pred, labels_test)

print "Accuracy score: ", accuracy_score_gnb
print "Precision score: ", precision_score_gnb
print "Recall score: ", recall_score_gnb

# Dumping classifier, my_dataset and features_list as .pkl files to be used in tester.py
dump_classifier_and_data(clf, my_dataset, features_list)

Accuracy score:  0.886363636364
Precision score:  0.0
Recall score:  0.0


In [246]:
clf_gbrt = GradientBoostingClassifier(random_state=42, 
                                      learning_rate=0.042548288288288286,
                                      min_samples_split=10,
                                     min_samples_leaf=1)
parameters = {'learning_rate':np.linspace(0.1, 0.00001, num=1000),
              'n_estimators':np.arange(1, 100, 1),
              'max_depth':np.arange(1, 13, 1),
              'max_features':np.arange(1, 20, 1),
              'min_samples_split':np.arange(2, 20, 1),
              'min_samples_leaf':np.arange(1, 20, 1)
             }
clf_gbrt_rscv = RandomizedSearchCV(clf_gbrt, param_distributions=parameters, cv=70, n_iter=100, scoring='precision', n_jobs=-1, verbose=2)
clf_gbrt_rscv.fit(features_train_scaled, labels_train)

print clf_gbrt_rscv.best_params_
print clf_gbrt_rscv.best_score_

# {'learning_rate': 0.042548288288288286, 'min_samples_leaf': 1, 'n_estimators': 99, 'max_features': 3, 'min_samples_split': 10, 'max_depth': 1}
# 0.06

Fitting 70 folds for each of 100 candidates, totalling 7000 fits
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV]  learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9, total=   0.1s


/Users/ammarjawaddoosh/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 


/Users/ammarjawaddoosh/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/ammarjawaddoosh/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9, total=   0.2s
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 
[CV]  learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9, total=   0.2s
[CV] learning_rate=0.027835045045, min_samples_leaf=1, n_estimators=83, min_samples_split=19, max_features=8, max_depth=9 


...

[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:    4.1s


[CV]  learning_rate=0.0254328828829, min_samples_leaf=8, n_estimators=55, min_samples_split=3, max_features=2, max_depth=4, total=   0.1s
[CV] learning_rate=0.0254328828829, min_samples_leaf=8, n_estimators=55, min_samples_split=3, max_features=2, max_depth=4 
[CV]  learning_rate=0.0254328828829, min_samples_leaf=8, n_estimators=55, min_samples_split=3, max_features=2, max_depth=4, total=   0.1s
[CV] learning_rate=0.0254328828829, min_samples_leaf=8, n_estimators=55, min_samples_split=3, max_features=2, max_depth=4 
[CV]  learning_rate=0.0254328828829, min_samples_leaf=8, n_estimators=55, min_samples_split=3, max_features=2, max_depth=4, total=   0.1s
[CV] learning_rate=0.0254328828829, min_samples_leaf=8, n_estimators=55, min_samples_split=3, max_features=2, max_depth=4 
[CV]  learning_rate=0.0254328828829, min_samples_leaf=8, n_estimators=55, min_samples_split=3, max_features=2, max_depth=4, total=   0.1s
[CV]  learning_rate=0.0254328828829, min_samples_leaf=8, n_estimators=55, min_s

[Parallel(n_jobs=-1)]: Done 872 tasks      | elapsed:    9.7s


[CV] learning_rate=0.00011009009009, min_samples_leaf=5, n_estimators=64, min_samples_split=6, max_features=3, max_depth=1 
[CV]  learning_rate=0.00011009009009, min_samples_leaf=5, n_estimators=64, min_samples_split=6, max_features=3, max_depth=1, total=   0.0s
[CV] learning_rate=0.0946952252252, min_samples_leaf=14, n_estimators=87, min_samples_split=12, max_features=14, max_depth=4 
[CV]  learning_rate=0.00011009009009, min_samples_leaf=5, n_estimators=64, min_samples_split=6, max_features=3, max_depth=1, total=   0.0s
[CV] learning_rate=0.00011009009009, min_samples_leaf=5, n_estimators=64, min_samples_split=6, max_features=3, max_depth=1 
[CV]  learning_rate=0.00011009009009, min_samples_leaf=5, n_estimators=64, min_samples_split=6, max_features=3, max_depth=1, total=   0.0s
[CV]  learning_rate=0.0946952252252, min_samples_leaf=14, n_estimators=87, min_samples_split=12, max_features=14, max_depth=4, total=   0.1s
[CV] learning_rate=0.0946952252252, min_samples_leaf=14, n_estimator

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/ammarjawaddoosh/anaconda2/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/Users/ammarjawaddoosh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

....................

In [177]:
#######
# SVC #
#######

clf = SVC(random_state=42, kernel='sigmoid', C=57.9, gamma=0.059, class_weight='balanced')

# Scaling both train and test features skipping 'poi' since it's a bool and the rest are financial values
min_max_scaler = MinMaxScaler()
features_train_scaled = min_max_scaler.fit_transform(features_train.iloc[:, 1:])
features_test_scaled = min_max_scaler.fit_transform(features_test.iloc[:, 1:])


# params = {'C': scipy.stats.expon(scale=100), 
#          'gamma': scipy.stats.expon(scale=.1), 
#          'kernel':['rbf', 'linear', 'poly', 'sigmoid'],
#          'class_weight': [None, 'balanced']
#         }
# clf_svc_rscv = RandomizedSearchCV(clf, param_distributions=params, cv=70, n_iter=50, scoring='precision', verbose=2, n_jobs=-1)
# clf_svc_rscv.fit(features_train_scaled, labels_train)
clf.fit(features_train_scaled, labels_train)

# pred = clf.predict(features_test_scaled)

# Classifier scores
precision_score_svc = precision_score(pred, labels_test)
recall_score_svc = recall_score(pred, labels_test)
accuracy_score_svc = accuracy_score(pred, labels_test)

print "Accuracy score: ", accuracy_score_svc
print "Precision score: ", precision_score_svc
print "Recall score: ", recall_score_svc

# print clf_svc_rscv.best_params_
# print clf_svc_rscv.best_score_

# parameters that return the best recall score:
# {'kernel': 'sigmoid', 'C': 22.124794209078111, 'gamma': 0.38710403019966072, 'class_weight': 'balanced'}
# 0.33

# parameters that return the best precision score:
# {'kernel': 'sigmoid', 'C': 57.914218875973347, 'gamma': 0.059626099388516422, 'class_weight': 'balanced'}
# 0.235

# Dumping classifier, my_dataset and features_list as .pkl files to be used in tester.py
dump_classifier_and_data(clf, my_dataset, features_list)

Accuracy score:  0.886363636364
Precision score:  0.4
Recall score:  0.5


In [161]:
#################
# Random Forest #
#################

# Fitting the model
clf = RandomForestClassifier(random_state=42, n_estimators=1, min_samples_leaf=2, min_samples_split=2, max_depth=31)

# RandomizedsearchCV to find optimal hyper params
#params = {
#          'n_estimators':np.arange(1, 5, 1),
#          'min_samples_leaf':np.arange(1, 5, 1),
#          'min_samples_split':np.arange(2, 20, 1),
#          'max_depth':np.arange(1, 40, 1),
#         }

#clf_rf_rscv = RandomizedSearchCV(clf, cv=70, n_iter=50, param_distributions=params, scoring='recall', verbose=2, n_jobs=-1)
#clf_rf_rscv.fit(features_train, labels_train)

clf.fit(features_train, labels_train)


# print clf_rf_rscv.best_params_
# print clf_rf_rscv.best_score_

# Dumping classifier, my_dataset and features_list as .pkl files to be used in tester.py
dump_classifier_and_data(clf, my_dataset, features_list)

# best results from randomizedsearchCV using "recall" scoring:
# {'n_estimators': 1, 'min_samples_split': 2, 'max_depth': 31, 'min_samples_leaf': 2}
# 0.21

{'n_estimators': 1, 'min_samples_split': 5, 'max_depth': 34, 'min_samples_leaf': 1}
0.21


# Q&A


1.Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]

>Machine learning is powerful at predicting whether a certain outcome is likely to happen (classification) or continuous numbers (regression). In this example where we are asked to predict that a person is a POI (classification) and we have features such as salary, bonus, stock (financial) as well as how many emails they have sent/received (count) we can use such features to learn if these help us predict whether people are POIs or not. As for outliers, I have only removed one due to the scarcity of data to begin with. The one I have removed is the column sum in the PDF of Enron employees' salaries because that's not a feature of an employee.

2.What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “intelligently select features”, “properly scale features”]

>As part of the EDA I looked at feature importances to understand which features were important and which weren't useful to include in my machine learning algorithms. I took all features which had an importance > 0, so that means the following 9 features: 'poi', 'salary', 'deferral_payments', 'total_payments','loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses'. As for the feature importances scores for the features I have used they are as follows:   
  1. feature: salary (0.220426513942)
  2. feature: deferral_payments (0.21197488041)
  3. feature: total_payments (0.132625994695)
  4. feature: loan_advances (0.106100795756)
  5. feature: bonus (0.105863661155)
  6. feature: restricted_stock_deferred (0.0757862826828)
  7. feature: deferred_income (0.0736811081639)
  8. feature: total_stock_value (0.0620731020005)
  9. feature: expenses (0.0114676611954)

> I have scaled the numerical features in my SVC model using MinMaxScaler to easier be able to compare the different financial measurements on a scale of 0 to 1.
> I have created my own feature bonus_salary_ratio which is bonus / salary based on the rationale that someone who has a high salary (top correlated feature with a POI) is likely also to have a high bonus however it made my models perform worse, especially GaussianNB due to the large amount of bonuses AND salaries which both were 0's so I was dividing 0 by 0, or some bonuses being negative leading to a -inf when divided by salary.

>As mentioned earlier I have decided to include any feature with a score > 0.

3.What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]

> I ended it up using the GaussianNB because it returned a precision of .4 and recall of .5. which was the highest scores out of them all. I tried SVC and Random Forest, but SVC performed the worse and I suspect it's due to the scarcity of data. I fine-tuned Random Forest using RandomizedSearchCV but couldn't get a precision above .3 for both precision and recall.

4.What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? What parameters did you tune? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric items: “discuss parameter tuning”, “tune the algorithm”]

> Tuning the parameters of an algorithm is really customising the model to match your dataset. For some of the parameters such as min_samples_split there is a trade-off between performance and accuracy: a high sample_split means you are avoiding over-fitting whereas a too high value means you are underfitting. Likewise with the learning_rate, if you give it too high a value you risk it might miss the optimal point whereas if it's too low it will take too long to converge and reach the local minima. For Random Forests, I've tuned the main parameters using RandomizedSearchCV but the best model didn't make it to the .3 precisio/recall scores. Below were the params and their ranges which I used for RandomizedSearchCV for Random Forest:

>params = {
          'n_estimators':np.arange(1, 5, 1),
          'min_samples_leaf':np.arange(1, 5, 1),
          'min_samples_split':np.arange(2, 20, 1),
          'max_depth':np.arange(1, 40, 1),
         }
         
> and for SVC I have used the following params:

> params = {'C': scipy.stats.expon(scale=100), 
          'gamma': scipy.stats.expon(scale=.1), 
          'kernel':['rbf', 'linear', 'poly', 'sigmoid'],
          'class_weight': [None, 'balanced']
         }
> if my GaussianNB classifier weren't the top performing model I would have fine-tuned my model using RandomizedSearchCV like I did for Random Forest and SVC to find the optimal parameters to use in my model.

5.What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric items: “discuss validation”, “validation strategy”]

> Validation, also known as cross-validation. It is used to prevent over-fitting to the training data. If you don't use cross-validation you risk overfitting your model to the training dataset which means it won't be able to perform well on un-seen new data because it is unable to generalise well. CV works by splitting the training dataset into smaller sets which the model is evaluated on and for each fold it will then return the score average accuracy score from all of the folds. CV is especially important with imbalanced classes as it increases the probability that your CV folds are more representative of the data.

6.Give at least 2 evaluation metrics and your average performance for each of them.  Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]
